<a href="https://colab.research.google.com/github/ruthbrennankk/scalable_group_project/blob/master/captcha_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
!pip3 install --index-url https://google-coral.github.io/py-repo/ tflite_runtime
!pip install captcha
!pip install stsci.ndimage

In [ ]:
# Imports
import os
import cv2
import numpy as np
import string
import random
import argparse
import tensorflow as tf
import tensorflow.keras as keras
import scipy.ndimage
import random
import captcha.image

from PIL import Image
import tflite_runtime.interpreter as tflite
import shutil
import pandas as pd

In [ ]:
captcha_symbols = "ABCDeFghijkMnPQRSTUVWXxYZz0123456789#/\[]:><%{}-+" #19

In [ ]:
import cv2
import numpy as np
import scipy.ndimage

def preprocess(raw_data) :
    return smallPreprocess(raw_data)

def lose_circles(i, cs):
    cs = cs[0]
    for c in cs:
        i = cv2.circle(i, (round(c[0]),round(c[1])), radius=round(c[2]), color=255, thickness=1)
    return i

def circles(i):
    cs = cv2.HoughCircles(i, method=cv2.HOUGH_GRADIENT, dp=1, minDist=1, param1=50, param2=5, minRadius=0, maxRadius=2)
    if cs is not None:
        i = lose_circles(i, cs)
    return i

def smallPreprocess(raw_data):
    img = cv2.cvtColor(raw_data, cv2.COLOR_BGR2RGB)
    image = np.array(img) / 255.0
    (c, h, w) = image.shape
    image = image.reshape([-1, c, h, w])
    return image

def pyPreprocess(raw_data):
    img = cv2.cvtColor(raw_data, cv2.COLOR_BGR2RGB)
    image = np.array(img, dtype=np.float32) / 255.0
    (c, h, w) = image.shape
    image = image.reshape([-1, c, h, w])
    return image

def smallPreprocess(raw_data):
    img = cv2.cvtColor(raw_data, cv2.COLOR_BGR2GRAY) # Gray Image
    img = cv2.erode(img, np.ones((2, 2), np.uint8), iterations=1)  # dilate image to initial stage (erode works similar to dilate because we thresholded the image the opposite way)
    img = cv2.dilate(img, np.ones((3, 3), np.uint8), iterations=1)  # erode just a bit to polish fine details
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB) # Back to Colour for channel
    image = np.array(img) / 255.0
    (c, h, w) = image.shape
    image = image.reshape([-1, c, h, w])
    return image

def bigPreprocess(raw_data) :
    #   Back to Black
    img = cv2.cvtColor(raw_data, cv2.COLOR_BGR2GRAY)
    # First Pass
    img = ~img  # invert
    img = cv2.erode(img, np.ones((2, 2), np.uint8), iterations=1)  # weaken noise
    img = ~img  # re-invert
    img = scipy.ndimage.median_filter(img, (5, 1))  # target lines
    img = scipy.ndimage.median_filter(img, (1, 3))  # target circles
    img = cv2.erode(img, np.ones((2, 2), np.uint8), iterations=1)
    img = scipy.ndimage.median_filter(img, (3, 3))  # target weak noise
    img = circles(img)  # Use Hough Transform on remaining circles
    # Last Pass
    img = cv2.dilate(img, np.ones((3, 3), np.uint8), iterations=1)  # actually performs erosion
    img = scipy.ndimage.median_filter(img, (5, 1))  # finally completely remove any extra noise that remains
    img = cv2.erode(img, np.ones((3, 3), np.uint8), iterations=2)  # dilate image to make it look like the original
    img = cv2.dilate(img, np.ones((3, 3), np.uint8), iterations=1)  # erode just a bit to polish fine details
    #Edge Detection
    # img = cv2.Canny(img, 100, 200)
    # Back to Colour
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    # Format - (different for pi)
    image = np.array(img) / 255.0
    (c, h, w) = image.shape
    return image.reshape([-1, c, h, w])

In [ ]:
# Generate
#!/usr/bin/env python3
def generate(count, output_dir, outputFile):
    width = 128
    height = 64
    captcha_symbols_g = captcha_symbols
    captcha_generator = captcha.image.ImageCaptcha(width=width, height=height)
    print("Generating captchas with symbol set {" + captcha_symbols_g + "}")

    if not os.path.exists(output_dir):
        print("Creating output directory " + output_dir)
        os.makedirs(output_dir)

    filenames = []
    index = 0
  
    for i in range(count):
        caplength = np.random.randint(1,7)
        random_str = ''.join([random.choice(captcha_symbols_g) for j in range(caplength)])

        img_name = 'img_'+str(i)+'.png'
        image_path = os.path.join(output_dir, img_name)
        if os.path.exists(image_path):
            version = 1
            while os.path.exists(os.path.join(output_dir, random_str + '_' + str(version) + '.png')):
                version += 1
            image_path = os.path.join(output_dir, random_str + '_' + str(version) + '.png')

        image = np.array(captcha_generator.generate_image(random_str))
        cv2.imwrite(image_path, image)

        with open(outputFile,'+a') as f:
            f.write(img_name + "," + random_str + "\n")

In [ ]:
# Calls to Generate  
generate(192000, '/content/train_set', '/content/train_labels.txt') # Train Set
generate(19200, '/content/val_set', '/content/val_labels.txt')  # Validation Set
generate(100, '/content/test_set', '/content/test_labels.txt')  # Test Set

In [ ]:
# Train
#!/usr/bin/env python3

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Build a Keras model given some parameters
def create_model(captcha_length, captcha_num_symbols, input_shape, model_depth=5, module_size=2):
  input_tensor = keras.Input(input_shape)
  x = input_tensor
  for i, module_length in enumerate([module_size] * model_depth):
      for j in range(module_length):
          x = keras.layers.Conv2D(32*2**min(i, 3), kernel_size=3, padding='same', kernel_initializer='he_uniform')(x)
          x = keras.layers.BatchNormalization()(x)
          x = keras.layers.Activation('relu')(x)
      x = keras.layers.MaxPooling2D(2)(x)

  x = keras.layers.Flatten()(x)
  x = [keras.layers.Dense(captcha_num_symbols, activation='softmax', name='char_%d'%(i+1))(x) for i in range(captcha_length)]
  model = keras.Model(inputs=input_tensor, outputs=x)

  return model

# A Sequence represents a dataset for training in Keras
# In this case, we have a folder full of images
# Elements of a Sequence are *batches* of images, of some size batch_size
class ImageSequence(keras.utils.Sequence):
    def __init__(self, directory_name, label_file, batch_size, captcha_length, captcha_symbols, captcha_width, captcha_height):
        self.directory_name = directory_name
        self.batch_size = batch_size
        self.captcha_length = captcha_length
        self.captcha_symbols = captcha_symbols
        self.captcha_width = captcha_width
        self.captcha_height = captcha_height
        with open(label_file,"+r") as f:
          labelList = f.readlines()
          self.labels = dict(zip(map(lambda x:x.split(",")[0],labelList),map(lambda x:x.split(",")[1].strip(),labelList)))
        file_list = os.listdir(self.directory_name)
        self.files = dict(zip(map(lambda x: x.split('.')[0], file_list), file_list))
        self.used_files = []
        self.count = len(file_list)
        # print('dir name ' + self.directory_name)
        # print(self.count)

    def __len__(self):
        return int(np.floor(self.count / self.batch_size))

    def __getitem__(self, idx):
        X = np.zeros((self.batch_size, self.captcha_height, self.captcha_width, 3), dtype=np.float32)
        y = [np.zeros((self.batch_size, len(self.captcha_symbols)), dtype=np.uint8) for i in range(self.captcha_length)]

        for i in range(min(self.batch_size, len(self.files))):
            random_image_name = random.choice(list(self.labels.keys()))
            random_image_label = self.labels[random_image_name]
            random_image_file = self.files[random_image_name[:-4]]

            # We've used this image now, so we can't repeat it in this iteration
            self.used_files.append(self.files.pop(random_image_name[:-4]))
            self.labels.pop(random_image_name)

            # We have to scale the input pixel values to the range [0, 1] for
            # Keras so we divide by 255 since the image is 8-bit RGB
            raw_data = cv2.imread(os.path.join(self.directory_name, random_image_file))
            processed_data = preprocess(raw_data)
            X[i] = processed_data

            if len(random_image_label) < self.captcha_length :
              balance = self.captcha_length - len(random_image_label)
              for x in range(0, balance) :
                random_image_label = random_image_label + ' '
            
            for j, ch in enumerate(random_image_label):
                y[j][i, :] = 0
                y[j][i, self.captcha_symbols.find(ch)] = 1

        return X, y

def train():
    # Inputs
    width = 128
    height = 64
    length = 6 
    batch_size = 32
    epochs = 2 #8
    train_dataset = '/content/train_set/'
    train_labels_file = '/content/train_labels.txt'
    validate_dataset = '/content/val_set/'
    val_labels_file = '/content/val_labels.txt'
    output_model_name = '/content/model_19_e7'
    input_model = '/content/model_19_e5' 
    captcha_symbols_t = captcha_symbols + ' '

    with tf.device('/device:GPU:0'):
        model = create_model(length, len(captcha_symbols_t), (height, width, 3))
        if input_model is not None:
            model.load_weights(input_model+'.h5')

        model.compile(loss='categorical_crossentropy',
                      optimizer=keras.optimizers.Adam(1e-3, amsgrad=True),
                      metrics=['accuracy'])
        model.summary()

        training_data = ImageSequence(train_dataset, train_labels_file, batch_size, length, captcha_symbols_t, width, height)
        validation_data = ImageSequence(validate_dataset, val_labels_file, batch_size, length, captcha_symbols_t, width, height)

        callbacks = [keras.callbacks.EarlyStopping(patience=3),
                      # keras.callbacks.CSVLogger('log.csv'),
                      keras.callbacks.ModelCheckpoint(output_model_name+'.h5', save_best_only=False)]

        # Save the model architecture to JSON
        with open(output_model_name+".json", "w") as json_file:
            json_file.write(model.to_json())

        try:
            model.fit_generator(generator=training_data,
                                validation_data=validation_data,
                                epochs=epochs,
                                callbacks=callbacks,
                                use_multiprocessing=True)
        except KeyboardInterrupt:
            print('KeyboardInterrupt caught, saving current weights as ' + output_model_name+'_resume.h5')
            model.save_weights(output_model_name+'_resume.h5')

if __name__ == '__main__':
    train()

In [ ]:
#   Classify
#!/usr/bin/env python3

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

def decode(characters, y):
    y = np.argmax(np.array(y), axis=2)[:,0]
    return ''.join([characters[x] for x in y])

def main():
    model_name = '/content/model_19_e7'
    captcha_dir = '/content/test_set/'
    label_file = '/content/test_labels.txt'
    output = '/content/output.csv'

    labels = None
    with open(label_file,"+r") as f:
      labelList = f.readlines()
      labels = dict(zip(map(lambda x:x.split(",")[0],labelList),map(lambda x:x.split(",")[1].strip(),labelList)))

    captcha_symbols_c = captcha_symbols + ' '

    print("Classifying captchas with symbol set {" + captcha_symbols_c + "}")
    correct = 0
    incorrect = 0

    with tf.device('/cpu:0'):
        with open(output, 'w') as output_file:
            json_file = open(model_name+'.json', 'r')
            loaded_model_json = json_file.read()
            json_file.close()
            model = keras.models.model_from_json(loaded_model_json)
            model.load_weights(model_name+'.h5')
            model.compile(loss='categorical_crossentropy',
                          optimizer=keras.optimizers.Adam(1e-3, amsgrad=True),
                          metrics=['accuracy'])
            
            for x in os.listdir(captcha_dir):
                # load image and preprocess it
                raw_data = cv2.imread(os.path.join(captcha_dir, x))
                image = preprocess(raw_data)

                prediction = model.predict(image)
                predictedAnswer = decode(captcha_symbols_c, prediction)
                output_file.write(x + ", " + predictedAnswer + "\n")
                predictedAnswer = predictedAnswer.replace(" ", "")

                answer = labels.get(x)
                print('Classified ' + x + ' ' + answer + '///' + predictedAnswer)
                if (answer == predictedAnswer) :
                    correct = correct + 1
                    print('^ CORRECT ^')
                else :
                    incorrect = incorrect + 1

    print('correct = '+str(correct))
    print('incorrect ='+str(incorrect))


if __name__ == '__main__':
    main()

In [ ]:
#!/usr/bin/env python3
# Converter

import tensorflow as tf
import tensorflow.keras as keras

def main(model_name):
    with open(f'/content/{model_name}.json') as f:
        model = f.read()
    model = keras.models.model_from_json(model)
    model.load_weights(f'/content/{model_name}.h5')
    cvt = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite = cvt.convert()
    with open(f'/content/converted_{model_name}.tflite', 'wb') as f:
        f.write(tflite)

if __name__ == '__main__':
    model_name = 'model_19_e7'
    main(model_name)

In [ ]:
# Classify - Lite
#!/usr/bin/env python3
import os
import cv2
import numpy as np
import tflite_runtime.interpreter as tflite
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

def decode(characters, y):
    y = np.argmax(np.array(y), axis=1)
    #y = np.argmax(np.array(y), axis=2)[:,0]
    return ''.join([characters[x] for x in y])

def main():
    
    model_name = '/content/converted_model_19_e7.tflite'
    captcha_dir = '/content/drive/MyDrive/Ruth/brennar5_imgs'
    output = '/content/brennar5_m17_e6_a11.csv'
    captcha_symbols_c = captcha_symbols + ' '

# Pi Addresses
    # model_name = '/users/ugrad/brennar5/captcha_detection/converted_test.tflite'
    # captcha_dir = '/users/ugrad/brennar5/captcha_detection/testers_4'
    # output = '/users/ugrad/brennar5/captcha_detection/converted_4_output.csv'

    count = 0
    print("Classifying captchas with symbol set {" + captcha_symbols_c + "}")
    with open(output, 'w') as output_file:
        
        tf_interpreter = tflite.Interpreter(model_name)
        tf_interpreter.allocate_tensors()
        input_tf = tf_interpreter.get_input_details()
        output_tf = tf_interpreter.get_output_details()
        files = os.listdir(captcha_dir)
        files = sorted(files)

        for x in files:
            # Load & Preprocess Image - Currently to work with test
            raw_data = cv2.imread(os.path.join(captcha_dir, x))
            image = pyPreprocess(raw_data)
            
            tf_interpreter.set_tensor(input_tf[0]['index'],image)
            tf_interpreter.invoke()
            prediction = []
            for output_node in output_tf:
                prediction.append(tf_interpreter.get_tensor(output_node['index']))
            prediction = np.reshape(prediction,(len(output_tf),-1))
            predictedAnswer = decode(captcha_symbols_c, prediction)
            predictedAnswer.strip()
            predictedAnswer = predictedAnswer.replace(" ", "")
            output_file.write(x + "," + predictedAnswer + "\n")

            answer = x[:-4]            
            print('Classified (count ' + str(count) + ') ' + answer + '///' + predictedAnswer)
            count = count + 1
                
if __name__ == '__main__':
    main()